# Coseg exeperiment

Due to size limitation, we only include the super pixels version (which is in the supplemantry), also, we only display 1 frame, as the translation from super pixels to pixel is memory heavy (3MB per frame).

In [2]:
using Images
using Distributed
using NPZ


┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1278


#### Data Loading and preprocessing

In [4]:
data = Dict()
for i=1:50
    group_data = copy(npzread("DATA/coseg/frame" * string(i) * ".npy")')
    group_data[1:3,:] ./= 25
    group_data[4:5,:] ./= 250
    data[i] = group_data
end

pixels_to_super1 = npzread("DATA/coseg/pixels_to_super1.npy")
pixels_to_super2 = npzread("DATA/coseg/pixels_to_super2.npy")
pixels_to_super3 = npzread("DATA/coseg/pixels_to_super3.npy")

println(keys(data))

Any[2, 11, 39, 46, 25, 42, 29, 8, 20, 14, 31, 33, 18, 26, 35, 17, 49, 44, 4, 37, 45, 13, 30, 1, 47, 32, 50, 40, 7, 9, 43, 34, 3, 38, 36, 48, 12, 16, 21, 10, 19, 22, 6, 24, 28, 5, 23, 27, 41, 15]


In [7]:
Pkg.instantiate()

using Distributed
addprocs(4)
@everywhere using VersatileHDPMixtureModels

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for VersatileHDPMixtureModels [cb401d7d-c4f9-4284-af24-a1eca280e8e8]
│   exception = Required dependency JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819] failed to load from a cache file.
└ @ Base loading.jl:1042
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for VersatileHDPMixtureModels [cb401d7d-c4f9-4284-af24-a1eca280e8e8]
│   exception = Required dependency JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819] failed to load from a cache file.
└ @ Base loading.jl:1042
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for VersatileHDPMixtureModels [cb401d7d-c4f9-4284-af24-a1eca280e8e8]
│   exception = Required dependency JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819] failed to load from a cache file.
└ @ Base loading.jl:1042
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for VersatileHDPMixtureModels [cb401

In [8]:
global_hyper_params = niw_hyperparams(1.0,
    [47.56,45.40,27.82] ./ 25,
    9.0,
    [[0.8662817 0.78323282 0.41225376];
        [0.78323282 0.74170384 0.50340258];
        [0.41225376 0.50340258 0.79185577]]*1.0)

local_hyper_params = niw_hyperparams(1.0,
    [217.857,511.084] ./ 250,
    5.0,
    Matrix{Float64}(I, 2, 2)*0.2)

LoadError: UndefVarError: I not defined

In [ ]:
hdp, history = vhdp_fit(data,3,10.0,100.0,10.0,global_hyper_params,local_hyper_params,150)
color_means = [(x.cluster_params.cluster_params.distribution.μ*25)/255 for x in hdp.global_clusters]
glabels = create_global_labels(hdp.groups_dict[1])
img = zeros(size(pixels_to_super1,1),size(pixels_to_super1,2),3)
for i=1:size(pixels_to_super1,1)
    for j=1:size(pixels_to_super1,2)
        img[i,j,:] = color_means[glabels[(pixels_to_super1)[i,j]+1]]
    end
end
image1 = img[:,:,[3,2,1]]
image1 = permutedims(image1,[3,1,2])
glabels = create_global_labels(hdp.groups_dict[2])
img = zeros(size(pixels_to_super2,1),size(pixels_to_super2,2),3)
for i=1:size(pixels_to_super2,1)
    for j=1:size(pixels_to_super2,2)
        img[i,j,:] = color_means[glabels[(pixels_to_super2)[i,j]+1]]
    end
end
image2 = img[:,:,[3,2,1]]
image2 = permutedims(image2,[3,1,2])

glabels = create_global_labels(hdp.groups_dict[3])
img = zeros(size(pixels_to_super3,1),size(pixels_to_super3,2),3)
for i=1:size(pixels_to_super3,1)
    for j=1:size(pixels_to_super3,2)
        img[i,j,:] = color_means[glabels[(pixels_to_super3)[i,j]+1]]
    end
end
image3 = img[:,:,[3,2,1]]
image3 = permutedims(image3,[3,1,2])


In [ ]:
colorview(RGB,image1)

In [ ]:
colorview(RGB,image2)

In [ ]:
colorview(RGB,image3)